## 1.读原文件 

In [1]:
import csv

# 专利每行内容太多，导致无法读出来，设置解除limit
csv.field_size_limit(2000 * 2000 * 400)

with open('/ssd01/Codes/PersonalCodes/ZhangXianpeng/graduation_design/datas/original_data/machine/machine_data1/patent_1.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    # 跳过表头
    next(reader)
    
    # 逐行读取数据, 并将需要叙述存入列表中
    text_list = []
    
    # 建立文章标题到id的映射，用于后续提取文章。
    id2title_dic = {}
    index = 0
    for row in reader:
        id = '##id:'+ str(index)
        title = row[0]
        IPC = row[5]
        abstract = row[7]
        claim = row[8]
        description = row[9]
        
        # 将文章编号、标题、摘要和权利声明合并，并处理成标注需要的格式
        text_list.append(id)
        text_list.append(title)
        text_list.append(abstract)
        text_list.append(claim)
        # text_list.append(description)
        id2title_dic[id] = title
        
        index = index + 1 

In [2]:
print("文章的总篇数:" , len(text_list)/4)

for i in range(0, 5):
    print("文章示例：", text_list[i])

文章的总篇数: 1000.0
文章示例： ##id:0
文章示例： Clematis plant named ‘Evipo110’
文章示例： A new Clematis plant with a compact growth habit, profuse, white flowers, and continuous summer flowering. The variety successfully propagates from softwood cuttings and is suitable for cultivation in commercial nursery culture. This new and distinct variety has shown to be uniform and stable in the resulting generations from asexual propagation from vegetative cuttings.
文章示例： 1. A new and distinct variety of Clematis plant named ‘Evipo110’, substantially as described and illustrated, due to its abundant white flowers with good keepability, attractive long lasting foliage and compact growth, suitability for production from softwood cuttings in pots, durable flowers and foliage which make the variety suitable for distribution in the floral industry.
文章示例： ##id:1


In [3]:
# 将id2title_dic存储为jsonl文件
import json

with open("/ssd01/Codes/PersonalCodes/ZhangXianpeng/graduation_design/datas/original_data/machine/id和标题的对应关系.jsonl", "w", encoding="utf-8" ) as file:
    json.dump(id2title_dic, file)

## 2. 将读的内容分割成每个单词 

In [ ]:
import re
import string

token_list = []        # 列表中的每个元素都是一个单词
for content in text_list:
    # 判断是否为文章编号,是的话则不做处理
    if content.startswith('##'):
        token_list.append(content)
    else:
        word_list = content.split()
        for word in word_list:
            # 删除单词中杂乱的符号
            word = re.split(r'[ \t!@#$%^&*()—\-_+=\\/\'",;:<>?\[\]{}|`~]+', word)
            for token in word:
                if token != '':
                    # 删除划分后word的标点符号
                    if token.endswith('.'):
                        token_list.append(token[:-1])
                        token_list.append('.')
                    else:
                        token_list.append(token)


In [ ]:
token_list

## 3.将单词逐行写入txt文件 

In [ ]:
with open("/ssd01/Codes/PersonalCodes/ZhangXianpeng/graduation_design/datas/original_data/machine/machine_data1/patent_1.txt", "w",encoding='utf-8') as file:
    # 将列表中的单词逐行写入文件
    for i in range(len(token_list) - 1):
        if token_list[i + 1].startswith('##id:'):
            file.write(". O\n")  
        else:
            file.write(token_list[i] + " O\n")

## 4.修改小常语料格式 

In [ ]:
token_label_list = []
with open("/ssd01/Codes/PersonalCodes/ZhangXianpeng/graduation_design/datas/fineturn_data/mechanical_data/mechine_chang.txt", "r",encoding='utf-8') as file:
    lines = file.readlines()
    id = 1
    for i in range(len(lines)):
        if lines[i] == "\n":
            token_label_list.append(". O\n")
            token_label_list.append("##id:" + str(id) + " O\n")
            id = id + 1
        else:
            token_label_list.append(lines[i])
print("处理前的长度为：", len(lines))
print("处理后的长度为：", len(token_label_list))

In [ ]:
with open("/ssd01/Codes/PersonalCodes/ZhangXianpeng/graduation_design/datas/fineturn_data/mechanical_data/mechine_zhang.txt", "w",encoding='utf-8') as file:
    for i in range(len(token_label_list)):
        file.write(token_label_list[i])